In [29]:
import string
import math

In [6]:
def remove_punctuation(value):
    result = ""
    for c in value:
        # If char is not punctuation, add it to the result.
        if c not in string.punctuation:
            result += c
    return result

# adiciona as palavras no dicionário: voc

In [13]:
cfcs = ['cf74','cf75','cf76','cf77','cf78','cf79']
voc = {}
for i in range(0,6):
    arq = open('cfc/' + cfcs[i])
    documentos = arq.read()
    documentos = documentos.split('\n\n')
    
    for documento in documentos:
        if "" in documento:
            break
        if len(documento) < 5:
            #print('erro')
            continue
        else:
#             try:
#                 autor = documento[documento.index('\nAU')+3:documento.index('\nTI')].split()
#             except:
#                 autor = []
                
            try:
                title = documento[documento.index('\nTI')+3:documento.index('\nSO')].split()
            except:
                title = []
                
#             try:
#                 fonte = documento[documento.index('\nSO')+3:documento.index('\nMJ')].split()
#             except:
#                 fonte = []
                
            try:
                mj = documento[documento.index('\nMJ')+3:documento.index('\nMN')].split()
            except:
                mj = []
                
            try:
                mn = documento[documento.index('MN')+3:documento.index('\nAB')].split()
            except:
                mn = documento[documento.index('MN')+3:documento.index('\nEX')].split() 
                
            try:
                try:
                    ab = documento[documento.index('\nAB')+3:documento.index('\nRF')].split()
                except:
                    ab = documento[documento.index('\nAB')+3:documento.index('\nCT')].split()                  
            except:
                try:
                    ab = documento[documento.index('\nEX')+3:documento.index('\nRF')].split()
                except:
                    try:
                        ab = documento[documento.index('\nEX')+3:].split()
                    except:
                        ab = documento[documento.index('\nAB')+3:].split()

        
            for w in title + mj + mn + ab:
                #print(w.strip(string.punctuation).lower())
                w = w.strip(string.punctuation).lower()
                if not w.isdigit():
                    try:
                        voc[w][i][1] += 1
                    except:
                        voc[w] = []
                        voc[w] = [['cf74', 0],['cf75', 0],['cf76', 0],['cf77', 0],['cf78', 0],['cf79', 0]]
                        voc[w][i][1] += 1

# gera o arquivo invertido

In [15]:
arquivoInv = open('arqInv.dat','w')
for k in voc:
    linha = k
    for i in voc[k]:
        if not i[1] == 0:
            linha += ';' + i[0] + ' ' + str(i[1])
        else:
            voc[k].remove(i)
    arquivoInv.writelines(linha + '\n')
arquivoInv.close()

# calcula o idf de todas as palavras

In [40]:
idfs = {}
for x in voc:
    idfs[x] = math.log(6.0/len(voc[x]))

# TESTES

In [18]:
pergunta = 'What are the effects of calcium on the physical properties of mucus from CF patients?'.strip(string.punctuation).lower().split()
pergunta

['what',
 'are',
 'the',
 'effects',
 'of',
 'calcium',
 'on',
 'the',
 'physical',
 'properties',
 'of',
 'mucus',
 'from',
 'cf',
 'patients']

In [19]:
for w in pergunta:
    print(voc[w])

[['cf74', 3], ['cf75', 1], ['cf76', 4], ['cf77', 3], ['cf78', 5], ['cf79', 5]]
[['cf74', 109], ['cf75', 116], ['cf76', 166], ['cf77', 89], ['cf78', 112], ['cf79', 144]]
[['cf74', 1336], ['cf75', 1704], ['cf76', 1931], ['cf77', 1578], ['cf78', 1497], ['cf79', 1992]]
[['cf74', 11], ['cf75', 19], ['cf76', 26], ['cf77', 24], ['cf78', 19], ['cf79', 28]]
[['cf74', 1252], ['cf75', 1531], ['cf76', 1791], ['cf77', 1420], ['cf78', 1346], ['cf79', 1829]]
[['cf74', 16], ['cf75', 4], ['cf76', 45], ['cf77', 8], ['cf78', 24], ['cf79', 17]]
[['cf74', 96], ['cf75', 110], ['cf76', 132], ['cf77', 112], ['cf78', 104], ['cf79', 152]]
[['cf74', 1336], ['cf75', 1704], ['cf76', 1931], ['cf77', 1578], ['cf78', 1497], ['cf79', 1992]]
[['cf74', 7], ['cf75', 4], ['cf76', 10], ['cf77', 9], ['cf78', 6], ['cf79', 9]]
[['cf74', 8], ['cf75', 9], ['cf76', 13], ['cf77', 3], ['cf78', 13], ['cf79', 7]]
[['cf74', 1252], ['cf75', 1531], ['cf76', 1791], ['cf77', 1420], ['cf78', 1346], ['cf79', 1829]]
[['cf74', 12], ['cf75', 